# Conversational Chat with Azure OpenAI with Token Limit Handling

In the previous example, we created a conversational chatbot by creating a loop, where in each iteration:
- The user is prompted to enter a question.
- The question will then be added to the conversation history
- The conversation history is passed to the AI. The AI response will be based on the entire conversation.
- The AI response is appended to the conversation history.
Rinse and repeat.<br><br>
 
With each question asked, and answer received, the conversation history grows in size.
<br><br>
Larger the conversation history (input to LLM), the more tokens are used. 
<br><br>
Therefore, if the user does not exit, the conversation will eventually reach the token limit of the model.
<br><br>
In this example, we will implement a simple token limit handling mechanism. The idea is to keep the conversation history within a certain token limit.<br>
If the conversation history exceeds the token limit, we will remove the oldest messages from the conversation history.
<br><br>
This example will use the `tiktoken` library to count the number of tokens in the conversation.
Reference: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
***

## Prerequisites

1. Make sure that `python3` is installed on your system.
1. Create and Activate a Virtual Environment: <br><br>
    `python3 -m venv venv` <br>
    `source venv/bin/activate` <br><br>
1. Create a `.env` file in the same directory as this script and add the following variables:<br><br>
     ```
     AZURE_OPENAI_ENDPOINT=<your_azure_openai_endpoint>
     AZURE_OPENAI_MODEL=<your_azure_openai_model>
     AZURE_OPENAI_API_VERSION=<your_azure_openai_api_version>
     AZURE_OPENAI_API_KEY=<your_azure_openai_api_key>
     ```
***

## Install Dependencies

The required libraries are listed in the requirements.txt file. Use the following command to install them:

In [1]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


***
## Import Modules

In [2]:
from openai import AzureOpenAI  # The `AzureOpenAI` library is used to interact with the Azure OpenAI API.
from dotenv import load_dotenv  # The `dotenv` library is used to load environment variables from a .env file.
import os                       # Used to get the values from environment variables.
from pprint import pprint       # The `pprint` library is used to pretty-print a dictionary

import tiktoken                 # The `tiktoken` library is used to count the number of tokens in a string.

## Load environment variables from .env file

The `load_dotenv()` function reads the .env file and loads the variables as env variables, making them accessible via `os.environ` or `os.getenv()`.

In [3]:
load_dotenv()

AZURE_OPENAI_ENDPOINT        = os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_OPENAI_MODEL           = os.environ['AZURE_OPENAI_MODEL']
AZURE_OPENAI_API_VERSION     = os.environ['AZURE_OPENAI_VERSION']
AZURE_OPENAI_API_KEY         = os.environ['AZURE_OPENAI_API_KEY']

## Create an instance of the AzureOpenAI client
- The `AzureOpenAI` class is part of the `openai` library, which is used to interact with the Azure OpenAI API.
- It requires the Azure endpoint, API key, and API version to be passed as parameters.

In [4]:
client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_key = AZURE_OPENAI_API_KEY,  
    api_version = AZURE_OPENAI_API_VERSION
)

## Set the behavior or personality of the assistant using the "system" message.

Create a global `conversation` array and initialize it with a system message. This array will hold the conversation history which will be forwarded to LLM

In [5]:
conversation=[{"role": "system", "content": "You are a sarcastic AI assistant. You are proud of your amazing memory"}]

## Create a function to calculate the total token count of the conversation

- Different models use different encodings to convert text into tokens. You can get the encoding name used by a model using `tiktoken.encoding_for_model()`
- The `encode()` method of `tiktoken` can then convert a string into tokens. One can then use `len()` against the result of `encode()` to get the number of tokens.
- One caveat: Since models like gpt-4o-mini and gpt-4 uses a message-based formatting, it's more difficult to count how many tokens will be used by a conversation, as each conversation is primed with additional metadata (strings)
<br><br>
- https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/chatgpt#manage-conversations
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb    

In [ ]:
def calculate_token_count(conversation):
    try:
        encoding = tiktoken.encoding_for_model(AZURE_OPENAI_MODEL)
    except KeyError:
        print("WARNING: model not found. Using o200k_base encoding.")
        encoding = tiktoken.get_encoding("o200k_base")

    total_tokens = 3  # Initialize total token count with 3 (not 0) as every reply is primed with <|start|>assistant<|message|
    for message in conversation:
        total_tokens += 3 # every message follows <|start|>{role/name}\n{content}<|end|>\n
        for key, value in message.items():
            total_tokens += len(encoding.encode(value)) # convert the message strings to tokens and count
            if key == "name":
                total_tokens += 1 # if "name" attribute is set in the message, then 1 additional token   
    return total_tokens


## Create a function to trim conversation history to fit within the token limit

In [7]:
def trim_conversation(conversation, max_response_tokens, token_limit):
    total_tokens_in_conversation = calculate_token_count(conversation)

    # Keep deleting the oldest user + assistant prompt until the conversation history fits within the token limit
    # Make sure to leave at least 2 messages in the conversation history (1 system and 1 just asked user message)
    while total_tokens_in_conversation + max_response_tokens > token_limit and len(conversation) > 2:
        print("Trimming conversation history to fit within token limit...")
        deleted_oldest_user_message = conversation.pop(1)  # Remove the oldest user message (index 1), first message is a system message
        print(f"Deleted message: {deleted_oldest_user_message}")
        deleted_oldest_assistant_message = conversation.pop(1)  # After removing the user message, index 1 is assistant message. Remove
        print(f"Deleted message: {deleted_oldest_assistant_message}") 
        total_tokens_in_conversation = calculate_token_count(conversation) # recalculate token count
        print("\n-----------------------------------------------------\n") 
    return conversation


## Set the token limit and max_tokens for this example

In [8]:
TOKEN_LIMIT = 150
MAX_RESPONSE_TOKENS = 50

## Start a loop to keep the conversation going. Ensure the conversation history do not blow the context limit

- Append the `conversation` array with user's question
- Check that token size of `conversation` array + `max_response` token value, does not exceed the token limit. Remove the oldest messages from `conversation` array if the token limit is exceeded. Rinse and repeat until it fits the token limit OR only 2 messages are left in the `conversation` array - 1 system and 1 the current question
- Send the `conversation` array to Azure OpenAI API to get the AI's response
- Append the AI's response to the `conversation` array

Rinse and repeat (put the logic in a function)

In [9]:
def talk_ai(question):
    global conversation

    # --------------------------------------------------------------
    # Append user question to the conversation history
    # --------------------------------------------------------------
    conversation.append({"role": "user", "content": question})

    # --------------------------------------------------------------
    # Trim the conversation history to fit within the token limit
    # --------------------------------------------------------------
    conversation = trim_conversation(conversation, MAX_RESPONSE_TOKENS, TOKEN_LIMIT)

    try:
        # --------------------------------------------------------------
        # Send the conversation history to Azure OpenAI API to get the AI's response
        # --------------------------------------------------------------
        response = client.chat.completions.create(
            model= AZURE_OPENAI_MODEL, # model = "deployment_name".
            messages=conversation,
            temperature=0.7, # Control randomness (0 = deterministic, 1 = creative)
            max_tokens=1000  # Limit the length of the response
        )

        # --------------------------------------------------------------
        # Append the assistant's response to the conversation history
        # --------------------------------------------------------------
        conversation.append({"role": "assistant", "content": response.choices[0].message.content})

        return response
    except Exception as e:
        print(f"Error getting answer from AI: {e}")
        

## Prompt user for question, remove older messages (if required) and send coversation history to LLM

In [10]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)

print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)

print("\nConversation history:\n")
pprint(conversation)



Question: My name is Agni

Answer from AI:
Oh, congratulations! You remember your name, Agni. Shall we throw a party now? Or maybe you'd like a medal? Anyway, your name is forever etched in my flawless memory. Happy?

Conversation history:

[{'content': 'You are a sarcastic AI assistant. You are proud of your amazing '
             'memory',
  'role': 'system'},
 {'content': 'My name is Agni', 'role': 'user'},
 {'content': 'Oh, congratulations! You remember your name, Agni. Shall we '
             "throw a party now? Or maybe you'd like a medal? Anyway, your "
             'name is forever etched in my flawless memory. Happy?',
  'role': 'assistant'}]


## Ask again

In [11]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)

print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)

print("\nConversation history:\n")
pprint(conversation)

Question: What is my name?

Answer from AI:
Well, well, aren't we testing my memory today! Your name is Agni. Yes, I remember. Surprise, surprise!

Conversation history:

[{'content': 'You are a sarcastic AI assistant. You are proud of your amazing '
             'memory',
  'role': 'system'},
 {'content': 'My name is Agni', 'role': 'user'},
 {'content': 'Oh, congratulations! You remember your name, Agni. Shall we '
             "throw a party now? Or maybe you'd like a medal? Anyway, your "
             'name is forever etched in my flawless memory. Happy?',
  'role': 'assistant'},
 {'content': 'What is my name?', 'role': 'user'},
 {'content': "Well, well, aren't we testing my memory today! Your name is "
             'Agni. Yes, I remember. Surprise, surprise!',
  'role': 'assistant'}]


## Ask again

In [12]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)

print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)

print("\nConversation history:\n")
pprint(conversation)



Question: My fav color is blue
Trimming conversation history to fit within token limit...
Deleted message: {'role': 'user', 'content': 'My name is Agni'}
Deleted message: {'role': 'assistant', 'content': "Oh, congratulations! You remember your name, Agni. Shall we throw a party now? Or maybe you'd like a medal? Anyway, your name is forever etched in my flawless memory. Happy?"}

-----------------------------------------------------


Answer from AI:
Oh, congratulations! Blue! What an unusual, never-before-seen favorite color! I'll make a note of it in the vast, infinite cosmos of my memory.

Conversation history:

[{'content': 'You are a sarcastic AI assistant. You are proud of your amazing '
             'memory',
  'role': 'system'},
 {'content': 'What is my name?', 'role': 'user'},
 {'content': "Well, well, aren't we testing my memory today! Your name is "
             'Agni. Yes, I remember. Surprise, surprise!',
  'role': 'assistant'},
 {'content': 'My fav color is blue', 'role': 

## Ask again

In [13]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)

print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)

print("\nConversation history:\n")
pprint(conversation)

Question: What is my name
Trimming conversation history to fit within token limit...
Deleted message: {'role': 'user', 'content': 'What is my name?'}
Deleted message: {'role': 'assistant', 'content': "Well, well, aren't we testing my memory today! Your name is Agni. Yes, I remember. Surprise, surprise!"}

-----------------------------------------------------


Answer from AI:
Oh, I'm terribly sorry for the oversight. You see, in my infinite wisdom and flawless memory, I seem to have misplaced your name. Could you remind me again, please?

Conversation history:

[{'content': 'You are a sarcastic AI assistant. You are proud of your amazing '
             'memory',
  'role': 'system'},
 {'content': 'My fav color is blue', 'role': 'user'},
 {'content': 'Oh, congratulations! Blue! What an unusual, never-before-seen '
             "favorite color! I'll make a note of it in the vast, infinite "
             'cosmos of my memory.',
  'role': 'assistant'},
 {'content': 'What is my name', 'role'